A good tool makes simple things easy and complex things possible.

If you don't know which solution is better, start with the one that's simpler.

In [ ]:
import os
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader

data = load(open('env/local-env.yml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})

from hemlock import push_app_context

app = push_app_context()

In [2]:
from hemlock import Input, Label, Page, Submit as S, Validate as V

N_ROUNDS = 5
POT = 20

def gen_proposal_input(round_):
    return Input(
        '''
        <p><b>Round {} of {}</b></p>
        <p>You have ${} to split between you and the responder. How 
        much money would you like to offer to the responder?</p>
        '''.format(round_, N_ROUNDS, POT),
        prepend='$', append='.00', var='Proposal',
        validate=V.range_val(0, POT),
        submit=S.data_type(int)
    )

path = Page(gen_proposal_input(1)).preview()

In [3]:
import random

response_input = Input(var='Response', data=5)
response_inputs = Input.query.filter(
    Input.var=='Response', Input.data!=None
).all()
random.choice(response_inputs).data

5

In [4]:
from hemlock import Compile as C, Embedded

from random import randint

@C.register
def proposer_outcome(outcome_label, proposal_input):
    # get the proposal
    proposal = POT-proposal_input.data, proposal_input.data
    # get all responses
    response_inputs = Input.query.filter(
        Input.var=='Response', Input.data!=None
    ).all()
    if response_inputs:
        # randomly choose a response
        response = random.choice(response_inputs).data
    else:
        # no responses are available
        # e.g. if this is the first participant
        response = randint(0, POT)
    # compute the payoff
    accept = response <= proposal[1]
    payoff = proposal if accept else (0, 0)
    # record results as embedded data
    outcome_label.page.embedded = [
        Embedded('Response', response),
        Embedded('Accept', int(accept)),
        Embedded('ProposerPayoff', payoff[0]),
        Embedded('ResponderPayoff', payoff[1])
    ]
    # describe the outcome of the round
    outcome_label.label = '''
        <p>You proposed the following split:</p>
        <ul>
            <li>You: ${}</li>
            <li>Responder: ${}</li>
        </ul>
        <p>The responder said they will accept any proposal which gives
        them at least ${}.</p>
        <p><b>Your proposal was {}, giving you a payoff of ${}.</b></p>
    '''.format(
        *proposal, response, 'accepted' if accept else 'rejected', payoff[0]
    )

proposal_outcome_page = Page(
    Label(
        compile=C.proposer_outcome(Input(data=10))
    )
)
proposal_outcome_page._compile().preview()

<Page 2>

In [5]:
[(e.var, e.data) for e in proposal_outcome_page.embedded]

[('Response', 5),
 ('Accept', 1),
 ('ProposerPayoff', 10),
 ('ResponderPayoff', 10)]

In [6]:
[os.remove(tmpfile) for tmpfile in app.tmpfiles]

[None, None]